#Aula 5: Modelo de Regressão e Machine Learning

##Introdução 
Chegou a hora de criar seu primeiro modelo de Machine Learning em nosso curso. 

Antes de começar execute os códigos da aula anterior, pois será necessário para essa aula. Feita a execução, continue.


In [0]:
import pandas as pd
import seaborn as sns
%precision %.2f
pd.options.display.float_format = '{:,.2f}'.format
uri = "https://github.com/guilhermesilveira/enem-2018/blob/master/MICRODADOS_ENEM_2018_SAMPLE_43278.csv?raw=true"
dados = pd.read_csv(uri)

In [0]:
colunas_de_notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
dados_notas = dados[colunas_de_notas].dropna()
dados_notas.columns = ['ciencias_naturais', 'ciencias_humanas', 'linguagem_codigo', 'matematica', 'redacao']
corr = dados_notas.corr()
sns.heatmap(corr)

In [0]:
sns.pairplot(dados_notas)

##O Processo de Classificação para o Aprendizado de Máquina

Discutiremos o que é o processo de classificação e como as máquinas podem aprender esse processo. Após essa discussão iniciaremos o tratamento dos dados para criar nosso primeiro modelo de ML.

A primeira coisa que devemos fazer é **separar os dados** que vamos usar como _entrada do nosso modelo_, dos que precisamos _prever_:



In [0]:
# Vamos usar as notas das provas de 
# ciências naturais, ciências humanas, matemática e redação 
# como entrada 
x_notas = dados_notas[['ciencias_naturais', 'ciencias_humanas', 'matematica', 'redacao']]
x_notas.head()

Como separamos os dados de entrada, também devemos fazer o mesmo com aqueles que _precisamos adivinhar_.

In [0]:
# Preveremos as notas da prova de linguagem e códigos.
y_adivinhar = dados_notas['linguagem_codigo']
y_adivinhar.head()

Agora temos os dados que precisamos classificar, mas repare que essa é toda nossa informação! Como assim?

Problema: Se eu treinar um modelo com todos esses dados, como eu vou conseguir medir a qualidade do modelo? 

Solução: Por isso precisamos dividir nossos dados em dois grupos, um para treino e outro para teste.

Para fazer isso vamos usar métodos da biblioteca [**Scikit-learn**](https://scikit-learn.org/stable/), ela é uma das **principais ferramentas** no mundo do Machine Learning! 

Além do Sklearn, iremos utilizar o **numpy** para setar o seed dos números pseudo-aleatórios.

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np

np.random.seed(43267)

#Ocorrera um divisao de x_notas e Y_adivinhar para essas variaveis
# O processo é o mesmo que realizar um função de f(x) = y
x_treino, x_teste, y_treino, y_teste = train_test_split(x_notas, y_adivinhar)

print(x_treino.shape)
print(y_treino.shape)

print(x_teste.shape)
print(y_teste.shape)

Feita a divisão dos nossos dados, chegou a hora de criar seu primeiro **Modelo de Regressão** (Em aula discutimos a diferença entre regressão e classificação).

## Modelo de Regressão

Vamos utilizar o método LinearSVR do scikit-learn:

In [0]:
#O programa ira aprender com os exemplos treino
from sklearn.svm import LinearSVR
modelo = LinearSVR()
modelo.fit(x_treino, y_treino)

Até o momento nós treinamos o modelo apenas com o `.fit()`, mas falta fazer a predição dos resultados. 

Para realizar a predição chamamos o método `.predict()` do modelo.


In [0]:
#Com a aprendizagem ele fará as previsões
predicoes_notas_linguagem = modelo.predict(x_teste)
predicoes_notas_linguagem[:4]

Você fez uma previsão do que seria y_teste analisando o x_teste e tendo como base  x_treino e y_treino. 

Agora compare a saída da predição com os valores reais logo abaixo, parece que está fazendo sentido, certo?!


In [0]:
# original_notas_linguagem
y_teste[:4]

##Plotando os Gráficos do Modelo de Regressão

Nos próximos trechos de códigos, vamos plotar alguns gráficos! 

As discussões e todas as análises sobre estas visualizações foram feitas de forma muito rica na videoaula, portanto recomendo fortemente acompanhá-las.


In [0]:
#Grafico dos valores previstos pelos os valores reais
import matplotlib.pyplot as plt
plt.figure(figsize = (9,9))
sns.scatterplot(x = y_teste.values, y = predicoes_notas_linguagem)

In [0]:
#Grafico dos valores previstos pelos erros de previsão
plt.figure(figsize = (9,9))
sns.scatterplot(x = y_teste.values, y = y_teste.values - predicoes_notas_linguagem)

In [0]:
#Previsão TOSCA usando como base a media das notas treinos
plt.figure(figsize = (9,9))
sns.scatterplot(x = y_teste.values, y = y_teste - y_treino.mean())

Após discutir esses gráficos, vamos criar mais um modelo de machine learning baseado em "árvores":


In [0]:
from sklearn.tree import DecisionTreeRegressor
modelo = DecisionTreeRegressor()
modelo.fit(x_treino, y_treino)
predicoes_notas_linguagem = modelo.predict(x_teste)

Após treinar o modelo de árvore e fazer as predições, plotamos duas imagens.

A **primeira** é muito parecida com as os gráficos do primeiro classificador.

In [0]:
#Grafico dos valores previstos pelos os valores reais com árvore
plt.figure(figsize = (7,7))
sns.scatterplot(x = y_teste.values, y = predicoes_notas_linguagem)

A **segunda mostra** os valores reais e valores previstos em relação as notas teste de matematica. Essa figura é muito interessante e mostra uma sobreposição muito boa entre elas, indicando que nossos resultados fazem sentido.



In [0]:
plt.figure(figsize = (9,9))
sns.scatterplot(x = x_teste['matematica'].values, y = predicoes_notas_linguagem)
sns.scatterplot(x = x_teste['matematica'].values, y = y_teste.values)

Avaliar os modelos por imagens é uma forma relevante, mas não resume a informação muito bem, por isso ficaria complexo avaliar dois ou três modelos apenas com gráficos.

##Métricas de Avaliação
O que precisamos agora é de uma métrica capaz de nos dizer como nosso modelo está indo, aqui vamos usar o **erro quadrático médio**. Existem centenas de métricas de avaliação, tudo vai depender do que você precisa e o que você está prevendo. 

In [66]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_teste, predicoes_notas_linguagem)

4156.93

Veja que nosso erro quadrático médio deu em torno dos 4186.22. 

Embora pelo gráfico nosso modelo pareça muito bom, pela métrica parece ser um pouco alto a margem de erros.

O MSE, sigla em inglês para essá métrica, é uma medida que **quanto mais perto de zero melhor**. 

Veja o resultado quando calculamos o MSE de dois vetores iguais:

In [0]:
# Nosso resultado é zero! 
mean_squared_error(y_teste, y_teste)

Você deve estar se perguntando: Meu modelo não está nem perto de zero, será que ele é tão ruim assim?

Nós ainda não temos como te dar essa resposta, precisamos de um critério comparativo, pois assim conseguimos dizer como nosso modelo está indo. 

Por exemplo, que tal classificar os nossos dados de uma maneira "bobinha", uma classificação de avaliação ingênua? Para isso temos os chamados métodos Dummy.

In [0]:
from sklearn.dummy import DummyRegressor
modelo_dummy = DummyRegressor()
modelo_dummy.fit(x_treino, y_treino)
dummy_predicoes = modelo_dummy.predict(x_teste)
mean_squared_error(y_teste, dummy_predicoes)

Finalmente conseguimos responder se nosso modelo é tão ruim assim! 

Na realidade nosso modelo não é um dos melhores, temos muito o que melhorar, mas já somos melhores que uma classificação ingênua. 

#Desafios

##Desafio 1
Explore os parâmetros C e o max_iter do modelo LinesSVR. Não há garantias que o resultado será melhor.

In [69]:
from sklearn.svm import LinearSVR
modelo_orig = LinearSVR()
modelo_orig.fit(x_treino, y_treino)
modelo = LinearSVR(C=0.00099, max_iter=72000)
modelo.fit(x_treino, y_treino)

predicoes_notas_linguagem_orig = modelo_orig.predict(x_teste)
print(predicoes_notas_linguagem_orig[:5])
predicoes_notas_linguagem = modelo.predict(x_teste)
print(predicoes_notas_linguagem[:5])

print("----------------------------------")
y_teste[:5]

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[540.94392095 648.38429838 535.41304744 536.42395167 595.1342355 ]
[474.39136846 545.42825734 486.75272814 421.59706174 543.5243963 ]
----------------------------------


41528   420.20
81918   545.60
22786   518.10
17935   539.20
70617   536.90
Name: linguagem_codigo, dtype: float64

In [0]:
plt.figure(figsize=(9,9))
sns.scatterplot(x=y_teste.values, y=y_teste - predicoes_notas_linguagem)

In [0]:
mean_squared_error(y_teste, predicoes_notas_linguagem_orig)

In [72]:
 mean_squared_error(y_teste, predicoes_notas_linguagem)

2323.14

##Desafio 2

Use a média das notas das 4 provas como input do input e plote um gráfico com a previsão e valores reais (Igual ao gráfico sugerido pelo Allan em aula)

In [0]:
plt.figure(figsize=(9,9))
sns.scatterplot(x=x_teste.mean(axis=1).values, y=predicoes_notas_linguagem)
sns.scatterplot(x=x_teste.mean(axis=1).values, y=y_teste.values)

##Desafio 3 

Remover as notas zero e testar os mesmos modelos, comparando o resultado

In [0]:
import numpy as np

dados_nota = dados.copy()
dados_nota_2 = dados_nota.copy()
dados_nota_sem_0 = dados_nota_2.copy()

dados_nota_sem_0['NU_NOTA_CN'] = dados_nota_2['NU_NOTA_CN'].replace(0., np.NAN)
dados_nota_sem_0['NU_NOTA_CH'] = dados_nota_2['NU_NOTA_CH'].replace(0., np.NAN)
dados_nota_sem_0['NU_NOTA_LC'] = dados_nota_2['NU_NOTA_LC'].replace(0., np.NAN)
dados_nota_sem_0['NU_NOTA_MT'] = dados_nota_2['NU_NOTA_MT'].replace(0., np.NAN)

dados_nota_sem_0.dropna(subset=['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT'], inplace=True)

dados_nota_sem_0 = dados_nota_sem_0[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT','NU_NOTA_REDACAO']]

In [0]:
dados_nota_sem_0.columns = ['ciencias_naturais', 'ciencias_humanas', 'linguagem_codigo', 'matematica', 'redacao']
x_4_notas_sem_nota_0 = dados_nota_sem_0[['ciencias_naturais', 'ciencias_humanas', 'matematica', 'redacao']]
x_4_notas_sem_nota_0.head()

In [0]:
y_adivinhar_sem_nota_0 = dados_nota_sem_0['linguagem_codigo']
y_adivinhar_sem_nota_0

In [0]:
np.random.seed(43267)

# f(x) = y
x_treino, x_teste, y_treino, y_teste = train_test_split(x_4_notas_sem_nota_0, y_adivinhar_sem_nota_0)

print(x_treino.shape)
print(x_teste.shape)
print(y_treino.shape)
print(y_teste.shape)

In [0]:
x_treino.head()

In [0]:
modelo_sem_0 = LinearSVR(C=0.000099, max_iter=72870)
modelo_sem_0.fit(x_treino, y_treino)

In [0]:
predicoes_notas_linguagem_sem_0 = modelo_sem_0.predict(x_teste)
predicoes_notas_linguagem_sem_0[:5]

In [0]:
plt.figure(figsize=(9,9))
sns.scatterplot(x=y_teste.values, y=predicoes_notas_linguagem_sem_0)

In [0]:
plt.figure(figsize=(9,9))
sns.scatterplot(x=x_teste.mean(axis=1).values, y=predicoes_notas_linguagem_sem_0)
sns.scatterplot(x=x_teste.mean(axis=1).values, y=y_teste.values)

In [0]:
mean_squared_error(y_teste, predicoes_notas_linguagem_sem_0)

##Desafio 4

Interpretar tudo que foi feito e compartilhar suas conclusões

In [0]:
Foi ensinado modelo de predições, onde através de algumas informações pode-se chegar a um valor aproximado para outra informação não realizada.
Também pude identificar que utilizando os parametros C e max_iter para situação testa da notas, pode se nota uma melhor aproximação desejada.

##Desafio 5

Calcule as métricas de erro que utilizamos (mean square root error) também no conjunto de treino, e veja o que acontece comparado com o conjunto de teste.

In [0]:
modelo = DecisionTreeRegressor()
modelo.fit(x_treino, y_treino)
predicoes_notas_linguagem = modelo.predict(x_treino)
mean_squared_error(y_treino, predicoes_notas_linguagem)